In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


In [14]:
tensor1 = torch.tensor([1,2,3])
tensor2 = torch.tensor([4,5,6])
print(tensor1 + tensor2)


tensor([5, 7, 9])


In [16]:
tensor1 = tensor1.to("cuda")
tensor2 = tensor2.to("cuda")

print(tensor1 + tensor2)

tensor([5, 7, 9], device='cuda:0')


In [17]:
tensor1 = tensor1.to("cpu")
tensor2 = tensor2.to("cuda")

print(tensor1)
print(tensor2)
print(tensor1 + tensor2)

tensor([1, 2, 3])
tensor([4, 5, 6], device='cuda:0')


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [20]:
x_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

x_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [25]:
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self, x, y):
        self.feetures = x
        self.labels = y
    def __getitem__(self, idx):
        return self.feetures[idx], self.labels[idx]
    
    def __len__(self):
        return self.labels.size()[0]

train_ds = ToyDataset(x_train, y_train)
test_ds = ToyDataset(x_test, y_test)

In [26]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    batch_size = 2,
    dataset=train_ds,
    shuffle = True,
    num_workers = 1,
    drop_last = True
)

test_loader =  DataLoader(
    batch_size = 2,
    dataset = test_ds,
    shuffle = False,
    num_workers = 1
    )

In [31]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, num_outputs)
        )
    
    def forward(self, x):
        return self.layers(x)

In [36]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(2, 2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)

        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train/Val Loss: {loss:.2f}")
    


Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [47]:
def compute_accuracy(model, dataloader, device):
    model = model.eval()

    total_samples = 0
    correct = 0.0

    for idx, (features, labels) in enumerate(dataloader):
        features, labels = features.to(device), labels.to(device)
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = predictions == labels
        correct += torch.sum(compare)
        total_samples += len(labels)
    
    print(correct)
    print(total_samples)
    return (correct / total_samples).item()

compute_accuracy(model, train_loader, device)

tensor(4., device='cuda:0')
4


1.0

In [46]:
compute_accuracy(model, test_loader, device)

1.0